In [1]:
import ee
ee.Initialize()

# Define Nairobi bounding box
nairobi = ee.Geometry.Rectangle([36.6, -1.5, 37.1, -1.1])

# Function to get Landsat image composite by year and sensor
def get_landsat_composite_all_bands(year):
    start = f'{year}-01-01'
    end = f'{year}-12-31'
    
    if year >= 2014:
        collection_id = 'LANDSAT/LC08/C02/T1_L2'
    elif year >= 2004:
        collection_id = 'LANDSAT/LE07/C02/T1_L2'
    else:
        collection_id = 'LANDSAT/LT05/C02/T1_L2'

    collection = ee.ImageCollection(collection_id) \
        .filterBounds(nairobi) \
        .filterDate(start, end) \
        .filter(ee.Filter.lt('CLOUD_COVER', 10))
    
    # Apply scaling to SR bands only, leave QA/thermal untouched
    def scale_sr(img):
        sr_bands = img.select(img.bandNames().filter(ee.Filter.stringContains('item', 'SR_')))
        scaled = sr_bands.multiply(0.0000275).add(-0.2)
        others = img.select(img.bandNames().removeAll(sr_bands.bandNames()))
        return scaled.addBands(others).copyProperties(img, img.propertyNames())
    
    processed = collection.map(scale_sr)
    return processed.median().clip(nairobi)

# List of years to export
years = [1984, 1989, 1994, 1999, 2004, 2009, 2014, 2019, 2023]

# Loop and export images
tasks = []
for year in years:
    image = get_landsat_composite_all_bands(year)
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'Nairobi_Landsat_{year}_AllBands',
        folder='earthengine',
        fileNamePrefix=f'nairobi_landsat_{year}_allbands',
        region=nairobi,
        scale=30,
        maxPixels=1e13
    )
    task.start()
    tasks.append((year, task))

# Optional: print task status
for year, task in tasks:
    print(f"Export task for {year}: {task.status()['state']}")


Export task for 1984: READY
Export task for 1989: READY
Export task for 1994: READY
Export task for 1999: READY
Export task for 2004: READY
Export task for 2009: READY
Export task for 2014: READY
Export task for 2019: READY
Export task for 2023: READY


In [2]:
import ee
ee.Initialize()

# Define Nairobi bounding box
nairobi = ee.Geometry.Rectangle([36.6, -1.5, 37.1, -1.1])

# Fallback configuration for the two exportable years
fallback_config = {
    1994: {
        'collection_id': 'LANDSAT/LT05/C02/T1_L2',
        'start': '1993-01-01',
        'end': '1995-12-31',
    },
    1999: {
        'collection_id': 'LANDSAT/LE07/C02/T1_L2',
        'start': '1999-05-01',
        'end': '1999-12-31',
    },
}

for year, config in fallback_config.items():
    collection = ee.ImageCollection(config['collection_id']) \
        .filterBounds(nairobi) \
        .filterDate(config['start'], config['end']) \
        .filter(ee.Filter.lt('CLOUD_COVER', 30))

    count = collection.size().getInfo()

    if count > 0:
        print(f"🛰️ Exporting fallback for {year} with {count} images")

        def scale_sr(img):
            sr_bands = img.select(img.bandNames().filter(ee.Filter.stringContains('item', 'SR_')))
            scaled = sr_bands.multiply(0.0000275).add(-0.2)
            others = img.select(img.bandNames().removeAll(sr_bands.bandNames()))
            return scaled.addBands(others).copyProperties(img, img.propertyNames())

        image = collection.map(scale_sr).median().clip(nairobi)

        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f'Nairobi_Landsat_{year}_fallback_allbands',
            folder='earthengine',
            fileNamePrefix=f'nairobi_landsat_{year}_fallback_allbands',
            region=nairobi,
            scale=30,
            maxPixels=1e13
        )
        task.start()
    else:
        print(f"⚠️ No valid images found for {year}. Skipping export.")

# Manual skip message for 1989
print("⚠️ Skipping 1989 fallback: No usable Landsat 5 imagery found for 1988–1990, even with relaxed filters.")


🛰️ Exporting fallback for 1994 with 1 images
🛰️ Exporting fallback for 1999 with 2 images
⚠️ Skipping 1989 fallback: No usable Landsat 5 imagery found for 1988–1990, even with relaxed filters.
